In [1]:
import os
import requests
from zipfile import ZipFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def download_file(url, save_path):
    with requests.get(url, stream=True) as r:
        with open(save_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

dataset_url = 'https://archive.ics.uci.edu/static/public/908/realwaste.zip'
download_dir = '/content/downloaded_dataset'

# Create the directory if it doesn't exist
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
zip_file_path = os.path.join(download_dir, 'dataset.zip')
download_file(dataset_url, zip_file_path)
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(download_dir)

In [2]:
import os
import numpy as np
from PIL import Image
from zipfile import ZipFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def random_patch(img, patch_size=(50, 50), random_probability=0.5):
    if np.random.rand() < random_probability:
        img_width, img_height = img.shape[1], img.shape[0]
        patch_x = np.random.randint(0, img_width - patch_size[1] + 1)
        patch_y = np.random.randint(0, img_height - patch_size[0] + 1)
        patch = img[patch_y:patch_y + patch_size[0], patch_x:patch_x + patch_size[1]]
        img_aug = img.copy()
        img_aug[patch_y:patch_y + patch_size[0], patch_x:patch_x + patch_size[1]] = np.random.randint(0, 256, patch_size + (img.shape[2],))
        return img_aug
    else:
        return img

# Function to perform data augmentation in batches
def augment_images_in_batches(dataset_dir, save_dir, batch_size=100, num_augmentations=2):
    rotation_range = np.random.randint(0, 180)  
    width_shift_range = np.random.uniform(0.0, 0.4)  
    height_shift_range = np.random.uniform(0.0, 0.4) 
    shear_range = np.random.uniform(0.0, 0.4)  
    zoom_range = np.random.uniform(0.8, 1.2) 

    horizontal_flip = np.random.choice([True, False])
    fill_mode = 'nearest'
    random_patch_probability = 0.5
    patch_size = (100, 100) 

    # Create an ImageDataGenerator with the randomly chosen augmentation settings
    datagen = ImageDataGenerator(
        rotation_range=rotation_range,
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip,
        fill_mode=fill_mode,
        preprocessing_function=lambda x: random_patch(x, patch_size=patch_size, random_probability=random_patch_probability)
    )

    subdirectories = [os.path.join(dataset_dir, d) for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

    for subdir in subdirectories:
        print(f"Augmenting images in {subdir}")
        save_subdir = os.path.join(save_dir, os.path.basename(subdir))
        if not os.path.exists(save_subdir):
            os.makedirs(save_subdir)
        image_files = [os.path.join(subdir, f) for f in os.listdir(subdir) if os.path.isfile(os.path.join(subdir, f))]
        num_batches = len(image_files) // batch_size
        for batch_index in range(num_batches):
            batch_files = image_files[batch_index * batch_size: (batch_index + 1) * batch_size]
            for image_file in batch_files:
                img = Image.open(image_file)  
                x = img.convert('RGB')  
                x = np.array(x)  
                x = x.reshape((1,) + x.shape) 
                i = 0
                for batch in datagen.flow(x, batch_size=1, save_to_dir=save_subdir, save_prefix='aug', save_format='jpg'):
                    i += 1
                    if i > num_augmentations:  
                        break  


dataset_dir = '/content/downloaded_dataset/realwaste-main/RealWaste'
save_dir = '/content/augmented_dataset'
augment_images_in_batches(dataset_dir, save_dir)

with ZipFile('/content/augmented_dataset2.zip', 'w') as zipf:
    for root, dirs, files in os.walk(save_dir):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), save_dir))

print("Data augmentation and zip file creation completed.")


Augmenting images in /content/downloaded_dataset/realwaste-main/RealWaste\Cardboard
Augmenting images in /content/downloaded_dataset/realwaste-main/RealWaste\Food Organics
Augmenting images in /content/downloaded_dataset/realwaste-main/RealWaste\Glass
Augmenting images in /content/downloaded_dataset/realwaste-main/RealWaste\Metal
Augmenting images in /content/downloaded_dataset/realwaste-main/RealWaste\Miscellaneous Trash
Augmenting images in /content/downloaded_dataset/realwaste-main/RealWaste\Paper
Augmenting images in /content/downloaded_dataset/realwaste-main/RealWaste\Plastic
Augmenting images in /content/downloaded_dataset/realwaste-main/RealWaste\Textile Trash
Augmenting images in /content/downloaded_dataset/realwaste-main/RealWaste\Vegetation
Data augmentation and zip file creation completed.


In [3]:
import os

# Function to generate a report containing the number of total samples in each directory
def generate_report(dataset_dir):
    # Get subdirectories in the dataset directory
    subdirectories = [os.path.join(dataset_dir, d) for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

    print("Report:")
    for subdir in subdirectories:
        # Count the number of image files in the subdirectory
        num_samples = len([f for f in os.listdir(subdir) if os.path.isfile(os.path.join(subdir, f))])

        print(f"Directory: {subdir}, Total Samples: {num_samples}")

# Define the directory where the dataset is located
dataset_dir = '/content/augmented_dataset/'

# Generate and print the report
generate_report(dataset_dir)


Report:
Directory: /content/augmented_dataset/Cardboard, Total Samples: 1137
Directory: /content/augmented_dataset/Food Organics, Total Samples: 1133
Directory: /content/augmented_dataset/Glass, Total Samples: 1119
Directory: /content/augmented_dataset/Metal, Total Samples: 1891
Directory: /content/augmented_dataset/Miscellaneous Trash, Total Samples: 1121
Directory: /content/augmented_dataset/Paper, Total Samples: 1389
Directory: /content/augmented_dataset/Plastic, Total Samples: 2361
Directory: /content/augmented_dataset/Textile Trash, Total Samples: 865
Directory: /content/augmented_dataset/Vegetation, Total Samples: 1112


In [8]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [9]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),  
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize pixel values
])
dataset = torchvision.datasets.ImageFolder(root='/content/downloaded_dataset/realwaste-main/RealWaste', transform=transform)
trainset, testset = train_test_split(dataset, test_size=0.25, random_state=42, shuffle=True)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=36, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=36, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network for %d images: %d %%' % (total, 100 * correct / total))

[1,    20] loss: 0.220
[1,    40] loss: 0.219
[1,    60] loss: 0.218
[1,    80] loss: 0.218
[2,    20] loss: 0.216
[2,    40] loss: 0.217
[2,    60] loss: 0.215
[2,    80] loss: 0.215
[3,    20] loss: 0.213
[3,    40] loss: 0.215
[3,    60] loss: 0.214
[3,    80] loss: 0.213
[4,    20] loss: 0.212
[4,    40] loss: 0.215
[4,    60] loss: 0.213
[4,    80] loss: 0.212
[5,    20] loss: 0.210
[5,    40] loss: 0.214
[5,    60] loss: 0.211
[5,    80] loss: 0.210
[6,    20] loss: 0.208
[6,    40] loss: 0.211
[6,    60] loss: 0.208
[6,    80] loss: 0.207
[7,    20] loss: 0.205
[7,    40] loss: 0.207
[7,    60] loss: 0.203
[7,    80] loss: 0.201
[8,    20] loss: 0.200
[8,    40] loss: 0.202
[8,    60] loss: 0.197
[8,    80] loss: 0.196
[9,    20] loss: 0.195
[9,    40] loss: 0.198
[9,    60] loss: 0.192
[9,    80] loss: 0.190
[10,    20] loss: 0.191
[10,    40] loss: 0.193
[10,    60] loss: 0.187
[10,    80] loss: 0.185
Finished Training
Accuracy of the network for 1188 images: 33 %


In [12]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),  
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize pixel values
])
dataset = torchvision.datasets.ImageFolder(root='/content/augmented_dataset/', transform=transform)
trainset, testset = train_test_split(dataset, test_size=0.25, random_state=42, shuffle=True)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=36, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=36, shuffle=False)

class Net_aug(nn.Module):
    def __init__(self):
        super(Net_aug, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net_aug = Net_aug()
criterion_aug = nn.CrossEntropyLoss()
optimizer_aug = optim.SGD(net_aug.parameters(), lr=0.01)
num_epochs = 10
for epoch in range(num_epochs):
    running_lossaug = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer_aug.zero_grad()
        outputs = net_aug(inputs)
        loss_aug = criterion_aug(outputs, labels)
        loss_aug.backward()
        optimizer_aug.step()
        running_lossaug += loss_aug.item()
        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_lossaug / 200))
            running_lossaug = 0.0

print('Finished Training')
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = net_aug(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (100 * correct / total))

[1,    20] loss: 0.219
[1,    40] loss: 0.219
[1,    60] loss: 0.219
[1,    80] loss: 0.218
[1,   100] loss: 0.218
[1,   120] loss: 0.217
[1,   140] loss: 0.217
[1,   160] loss: 0.217
[1,   180] loss: 0.217
[1,   200] loss: 0.216
[1,   220] loss: 0.215
[1,   240] loss: 0.217
[2,    20] loss: 0.216
[2,    40] loss: 0.216
[2,    60] loss: 0.216
[2,    80] loss: 0.216
[2,   100] loss: 0.214
[2,   120] loss: 0.212
[2,   140] loss: 0.214
[2,   160] loss: 0.215
[2,   180] loss: 0.215
[2,   200] loss: 0.214
[2,   220] loss: 0.213
[2,   240] loss: 0.216
[3,    20] loss: 0.214
[3,    40] loss: 0.215
[3,    60] loss: 0.215
[3,    80] loss: 0.215
[3,   100] loss: 0.213
[3,   120] loss: 0.210
[3,   140] loss: 0.213
[3,   160] loss: 0.214
[3,   180] loss: 0.213
[3,   200] loss: 0.212
[3,   220] loss: 0.212
[3,   240] loss: 0.214
[4,    20] loss: 0.212
[4,    40] loss: 0.212
[4,    60] loss: 0.213
[4,    80] loss: 0.212
[4,   100] loss: 0.210
[4,   120] loss: 0.207
[4,   140] loss: 0.210
[4,   160] 